In [1]:
import datetime

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from Project.Database import Db

In [2]:
year1_minute, meta = Db.load_data(hourly=False, meta=True, year=1)

In [3]:
year1_hourly, meta = Db.load_data(hourly=True, meta=True, year=1)

In [4]:
year2_hourly, meta = Db.load_data(hourly=True, meta=True, year=2)

In [5]:
year1_minute["Timestamp"] = pd.to_datetime(year1_minute["Timestamp"], format="%Y-%m-%d %H:%M:%S%z")

ValueError: Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True

In [ ]:
fig = px.scatter(year1_minute.loc[:10000], x="Timestamp", y="Load_DryerPowerTotal")
fig.show()

In [ ]:
consumption = meta.loc[((meta['Parameter'] == "Power_Electrical") | (meta['Parameter'] == "Power_Thermal")) & (
    meta["Description"].str.contains("power consumption" or "used"))]["Unnamed: 0"].tolist()

In [ ]:
appliance_instant_energy_consumed = pd.DataFrame(year1_hourly[consumption].sum(axis=0).sort_values(ascending=False),
                                                 columns=["Year 1"])

for col in appliance_instant_energy_consumed.index:
    appliance_instant_energy_consumed.loc[col, "Year 2"] = year2_hourly[col].sum() if col in year2_hourly.columns else 0

appliance_instant_energy_consumed = appliance_instant_energy_consumed.loc[
    (appliance_instant_energy_consumed.sum(1) > 0)]

In [ ]:
fig = px.bar(appliance_instant_energy_consumed, x=["Year 1", "Year 2"], barmode="group", log_x=True)
fig.show()

In [ ]:
plugs = meta.loc[(meta['Parameter'] == "Status_OnOff") & (meta["Measurement_Location"] == "Kitchen") & (
    meta["Unnamed: 0"].str.contains("Plug"))]["Unnamed: 0"].tolist()

In [ ]:
kitchenplugs = year1_hourly[["Timestamp"] + plugs].copy()
kitchenplugs["Status"] = kitchenplugs[plugs].sum(axis=1)
kitchenplugs

In [ ]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
kitchenplugs["Timestamp"] = pd.to_datetime(kitchenplugs["Timestamp"], format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(unit="h", arg=(kitchenplugs["Timestamp"].str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[1]).astype(int))

for index in kitchenplugs.index:
    kitchenplugs.loc[index, "Timestamp"] = kitchenplugs.loc[index, "Timestamp"] - datetime.timedelta(
        seconds=(int(str(year1_hourly.loc[index, "Timestamp"]).split("-")[-1][1])) * 3_600)

#Extracts hour of Timestamp.
kitchenplugs["HourOfTimestamp"] = kitchenplugs.Timestamp.dt.hour

#Replaces values to be binary 0 or 1.
for i in plugs:
    kitchenplugs.loc[kitchenplugs[i] < 0, i] = 0
    kitchenplugs.loc[kitchenplugs[i] > 0, i] = 1

#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for all attributes in kitchenplugs
hourON_kitchen = kitchenplugs.groupby(["HourOfTimestamp"])[plugs].sum()

def normalise_dataframe(df):
    '''
    Normalises values of all attributes in a dataframe, with minmax method.
    :param df:
    :type df:
    :return:
    :rtype:
    '''
    normalised_df = df.copy()

    for column in normalised_df:
        try:
            normalised_df[column] = (normalised_df[column] - normalised_df[column].min()) / (
                    normalised_df[column].max() - normalised_df[column].min())
        except:
            normalised_df[column] = normalised_df[column]

    return normalised_df

#set up data to heatmap
columns = hourON_kitchen.columns[1:].tolist()
hourON_kitchen[columns] = normalise_dataframe(hourON_kitchen[columns])
hourON_kitchen["HourOfTimestamp"] = hourON_kitchen.index
z_data = []
for i in hourON_kitchen[columns]:
    data_list = hourON_kitchen[i].round(2).values.tolist()
    z_data.append(data_list)

#Annotate z_values
annotations = go.Annotations()
for n, row in enumerate(z_data):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text="" if z_data[n][m] == 0 else str(z_data[n][m]),
                                         x=hourON_kitchen["HourOfTimestamp"][m], y=columns[n], xref='x1', yref='y1',
                                         showarrow=False))

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=hourON_kitchen["HourOfTimestamp"],
    y=columns,
    colorscale='Teal')
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30, annotations=annotations, )

fig.update_xaxes(title="Hour of day")
fig.update_yaxes(title="Appliance", showticklabels=True)
fig.show()

In [ ]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
year1_hourly["Timestamp"] = pd.to_datetime(year1_hourly["Timestamp"], format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(unit="h", arg=(year1_hourly["Timestamp"].str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[1]).astype(int))

#Extracts hour of Timestamp.
year1_hourly["HourOfTimestamp"] = year1_hourly.Timestamp.dt.hour

#Replaces values to be binary 0 or 1.
for i in consumption:
    year1_hourly.loc[year1_hourly[i] < 0, i] = 0
    year1_hourly.loc[year1_hourly[i] > 0, i] = 1

#Creates a dataframe with counts of "StatusOffOn" = 1, grouped by HourOfTimestamp for all attributes in year1_hours
hourON_all = year1_hourly.groupby(["HourOfTimestamp"])[consumption].sum()

#set up data to heatmap
columns = hourON_all.columns[1:].tolist()
hourON_all[columns] = normalise_dataframe(hourON_all[columns])
hourON_all["HourOfTimestamp"] = hourON_all.index
z_data = []
for i in hourON_all[columns]:
    data_list = hourON_all[i].round(2).values.tolist()
    z_data.append(data_list)

#Annotate z_values
annotations = go.Annotations()
for n, row in enumerate(z_data):
    for m, val in enumerate(row):
        annotations.append(go.Annotation(text="" if z_data[n][m] == 0 else str(z_data[n][m]),
                                         x=hourON_all["HourOfTimestamp"][m], y=columns[n], xref='x1', yref='y1',
                                         showarrow=False))

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=z_data,
    x=hourON_all["HourOfTimestamp"],
    y=columns,
    colorscale='Teal')
)

fig.update_layout(
    margin=dict(
        l=30,
        r=30,
        b=30,
        t=50, ),
    xaxis_nticks=30,
    annotations=annotations)

fig.update_xaxes(title="Hour of day")
fig.update_yaxes(title="Appliance", showticklabels=True)
fig.show()

In [ ]:
year1_minute